In [ ]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q scipy


In [54]:
from sentence_transformers import SentenceTransformer
SEmodel = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

In [55]:
from wikipediaapi import Wikipedia
wiki = Wikipedia('RAGBot/0.0', 'en')
doc = wiki.page('Satoru Gojo').text
paragraphs = doc.split('\n\n') # chunking

In [56]:
doc

'Satoru Gojo (Japanese: 五条 悟, Hepburn: Gojō Satoru) is a character from Gege Akutami\'s manga Jujutsu Kaisen. He was first introduced in Akutami\'s short series Tokyo Metropolitan Curse Technical School as the mentor of the cursed teenager Yuta Okkotsu at Tokyo Prefectural Jujutsu High School. This miniseries became the prequel Jujutsu Kaisen 0 of Jujutsu Kaisen. In Jujutsu Kaisen, Gojo takes the same role but mentors the student Yuji Itadori who suffers a similar Curse, helping him become stronger while protecting other characters in the series.\nGojo was designed by Akutami to be a formidable yet endearing figure who is passionate about his students. He is voiced by Yūichi Nakamura in Japanese and Kaiji Tang in English in the animated adaptations by MAPPA.\nThe character was well-received by the media for his carefree nature and power shown when protecting his students, becoming the series\'s breakout character. Furthermore, his role in the prequel Jujutsu Kaisen 0 was appreciated by

In [15]:
import textwrap
for i, p in enumerate(paragraphs):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")

-----------------------------------------------------------------
Satoru Gojo (Japanese: 五条 悟, Hepburn: Gojō Satoru) is a character from Gege Akutami's manga Jujutsu
Kaisen. He was first introduced in Akutami's short series Tokyo Metropolitan Curse Technical School
as the mentor of the cursed teenager Yuta Okkotsu at Tokyo Prefectural Jujutsu High School. This
miniseries became the prequel Jujutsu Kaisen 0 of Jujutsu Kaisen. In Jujutsu Kaisen, Gojo takes the
same role but mentors the student Yuji Itadori who suffers a similar Curse, helping him become
stronger while protecting other characters in the series. Gojo was designed by Akutami to be a
formidable yet endearing figure who is passionate about his students. He is voiced by Yūichi
Nakamura in Japanese and Kaiji Tang in English in the animated adaptations by MAPPA. The character
was well-received by the media for his carefree nature and power shown when protecting his students,
becoming the series's breakout character. Furthermore,

In [64]:
docs_embed = SEmodel.encode(paragraphs, normalize_embeddings=True)

In [65]:
docs_embed.shape

(9, 768)

In [68]:
docs_embed[0]

array([ 3.28321829e-02, -1.73554085e-02,  2.44257087e-03, -4.31342376e-03,
       -4.27367613e-02,  7.42530217e-03,  1.54507328e-02, -7.22436560e-03,
        1.08024487e-02, -2.15968098e-02, -1.09680174e-02, -1.27828838e-02,
        1.42038232e-02,  6.08282834e-02,  3.18820998e-02,  3.44335400e-02,
       -4.00627404e-02, -4.91126217e-02, -1.85463596e-02, -1.31067261e-02,
       -9.89671331e-03, -3.09945922e-02, -2.03995351e-02, -3.58905867e-02,
       -1.87303480e-02,  2.95224469e-02, -5.24713993e-02,  8.17153454e-02,
        1.60831977e-02, -5.10793701e-02,  2.85787694e-02,  1.47317573e-02,
        1.64737576e-03, -1.30452290e-02,  1.94885209e-02, -3.40623409e-02,
       -4.85156626e-02,  2.62024049e-02, -3.06329615e-02, -2.36394592e-02,
       -1.90322157e-02,  5.59457876e-02, -5.22277951e-02,  2.08166912e-02,
       -3.78269553e-02,  4.52065989e-02, -2.81005241e-02, -3.61495861e-03,
       -1.65178459e-02, -6.69185817e-02, -4.94142883e-02, -5.34475818e-02,
       -1.24388086e-02, -

In [72]:
query = "WHo is gojo"
query_embed = SEmodel.encode(query, normalize_embeddings=True)


In [74]:
query_embed.shape

(768,)

In [76]:
import numpy as np
similarities = np.dot(docs_embed, query_embed.T)

In [78]:
similarities.shape

(9,)

In [80]:
similarities

array([0.6789534 , 0.50833523, 0.5587791 , 0.42006588, 0.50006217,
       0.5220425 , 0.53133285, 0.19516248, 0.13984874], dtype=float32)

In [82]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()

In [84]:
top_3_idx


[0, 2, 6]

In [86]:
most_similar_documents = [paragraphs[idx] for idx in top_3_idx]

In [88]:
CONTEXT = ""
for i, p in enumerate(most_similar_documents):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")
  CONTEXT += wrapped_text + "\n\n"

-----------------------------------------------------------------
Satoru Gojo (Japanese: 五条 悟, Hepburn: Gojō Satoru) is a character from Gege Akutami's manga Jujutsu
Kaisen. He was first introduced in Akutami's short series Tokyo Metropolitan Curse Technical School
as the mentor of the cursed teenager Yuta Okkotsu at Tokyo Prefectural Jujutsu High School. This
miniseries became the prequel Jujutsu Kaisen 0 of Jujutsu Kaisen. In Jujutsu Kaisen, Gojo takes the
same role but mentors the student Yuji Itadori who suffers a similar Curse, helping him become
stronger while protecting other characters in the series. Gojo was designed by Akutami to be a
formidable yet endearing figure who is passionate about his students. He is voiced by Yūichi
Nakamura in Japanese and Kaiji Tang in English in the animated adaptations by MAPPA. The character
was well-received by the media for his carefree nature and power shown when protecting his students,
becoming the series's breakout character. Furthermore,

In [90]:
query = "Who is GOJO ?"

In [92]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [94]:
import google.generativeai as genai

genai.configure(api_key="GOOGLE_API_KEY")  
model = genai.GenerativeModel("gemini-1.5-flash")
messages = [
    {"role": "user", "content": prompt}
]
response = model.start_chat().send_message(prompt)
print(response.text)


Satoru Gojo is a character from Gege Akutami's manga *Jujutsu Kaisen*.  He is a mentor to the students Yuji Itadori and Yuta Okkotsu, a powerful and endearing figure known for his carefree nature and immense abilities.

